In [1]:
!pip install gdown
!gdown --folder "https://drive.google.com/drive/folders/1pxqwBw2nSzZmPG4_B0huwW7nJJ-GlcNa?usp=sharing"

Retrieving folder contents
Processing file 1dZ6mhxHvM_QvAdfSqWzb65U3zb2MY4uZ dev_canonical.csv
Processing file 1cwWovBZMYLHcwSr__2ocYx5VFpLQZ057 dev-00000-of-00001 (1).csv
Processing file 12HEI-zcyB2Yge1zkssjSxkRTzXUicPxi ipc_embeddings.npy
Processing file 1Cqned12VCLNTXPJPVGGJmdzaag9neEBc ipc_graph.pkl
Processing file 13m7_02-yGhs4SYEVCWydLrJ7NgYeFjdc ipc_sections.pkl
Processing file 1OEhEb10evvBqEmPbtXkZD_DbYlnj2O-f mlb_fixed.pkl
Processing file 1umpK4po91SlNXjsSBSUVAiSKC5Rtm10h Purushottam_Dashrath_Borate_vs_State_Of_Maharashtra_on_8_May_2015.PDF
Processing file 11FL5pyhEXLIweExurZTy_RAgC8KEdq97 statutes-00000-of-00001.csv
Processing file 1Mx58IzUN4AsLnFQ4T1yYpvNf7Ih3OFrf statutes.csv
Processing file 1u3I2b2voR5ZFAPmGWghjzf1cctXiBrQi test_canonical.csv
Processing file 1Qm8EJ_FVVSBoGnCdFg_7mfxY1ju8Nof5 test-00000-of-00001.csv
Processing file 1j7TA7Xt3GxgET9Hn4xN5sbDik9DADmsJ train_canonical_mlb_fixed.csv
Processing file 1hRxgbmJnrnEb-Hrai5uTMs64PQnrN75R train_canonical.csv
Processing

In [2]:
# Cell 0: (run only if you need to install/upgrade)
# NOTE: keep versions generic to avoid dependency conflicts on Kaggle.
# If transformers or torch are not present, uncomment and install. Usually Kaggle has them.
!pip install -q transformers torch pandas tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 6.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 8.9 MB/s eta 0:00:0000:0100:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [9]:
# ---------- Cell 0 (run first) ----------
# Monkeypatch to provide MessageFactory.GetPrototype for environments
# where protobuf removed that symbol. This avoids the cryptic AttributeError.
# Run this cell BEFORE importing transformers / sentence-transformers.

try:
    from google.protobuf.message_factory import MessageFactory
    # If GetPrototype is missing, create a shim that uses GetMessageClass (newer API)
    if not hasattr(MessageFactory, "GetPrototype"):
        # some protobuf versions provide GetMessageClass on the factory
        if hasattr(MessageFactory, "GetMessageClass"):
            def _get_prototype_shim(self, descriptor):
                # GetMessageClass returns python class for descriptor
                return self.GetMessageClass(descriptor)
            MessageFactory.GetPrototype = _get_prototype_shim
            print("Patched MessageFactory.GetPrototype -> GetMessageClass shim applied.")
        else:
            # fallback shim using descriptor_pool (less likely needed)
            from google.protobuf import descriptor_pool, message_factory, symbol_database
            _pool = descriptor_pool.Default()
            def _get_prototype_fallback(self, descriptor):
                # create a new message class via symbol_database
                db = symbol_database.Default()
                try:
                    # this may raise if not registered; attempt to register
                    return db.GetPrototype(descriptor)
                except Exception:
                    # last resort: use message_factory to create a prototype
                    return message_factory.MessageFactory().GetPrototype(descriptor)
            MessageFactory.GetPrototype = _get_prototype_fallback
            print("Patched MessageFactory.GetPrototype -> fallback shim applied.")
    else:
        print("MessageFactory.GetPrototype already present; no patch needed.")
except Exception as e:
    print("Could not apply protobuf MessageFactory patch (continuing). Exception:", e)
# ---------------------------------------

Patched MessageFactory.GetPrototype -> fallback shim applied.


In [10]:
# Cell 1 - imports & device
import os
import json
import math
import random
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer

print("torch version:", torch.__version__)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

torch version: 2.6.0+cu124
Device: cuda


In [11]:
# Cell 2 - helpers: clean, embedder
import re

def clean_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    s = s.replace("\u00A0", " ")
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Embedding function using transformers + mean-pooling
class TransformerEmbedder:
    def __init__(self, model_name: str = "distilbert-base-uncased", batch_size: int = 32):
        self.model_name = model_name
        self.batch_size = batch_size
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.to(DEVICE)
        self.model.eval()

    def embed_texts(self, texts: List[str], show_progress: bool = True) -> np.ndarray:
        """
        Return L2-normalized mean-pooled embeddings (numpy array shape (N, D))
        """
        all_embs = []
        bs = self.batch_size
        texts = [clean_text(t) for t in texts]
        for i in tqdm(range(0, len(texts), bs), disable=not show_progress, desc="Embedding"):
            batch = texts[i:i+bs]
            enc = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256)
            input_ids = enc["input_ids"].to(DEVICE)
            attention_mask = enc["attention_mask"].to(DEVICE)
            with torch.no_grad():
                out = self.model(input_ids=input_ids, attention_mask=attention_mask)
                # out.last_hidden_state: (B, L, D)
                hidden = out.last_hidden_state
                mask = attention_mask.unsqueeze(-1)
                summed = (hidden * mask).sum(dim=1)    # (B, D)
                lens = mask.sum(dim=1).clamp(min=1e-9) # (B,1)
                mean_pooled = summed / lens
                # convert to CPU numpy
                emb = mean_pooled.cpu().numpy()
                all_embs.append(emb)
        all_embs = np.vstack(all_embs) if all_embs else np.zeros((0, self.model.config.hidden_size))
        # L2-normalize
        norms = np.linalg.norm(all_embs, axis=1, keepdims=True) + 1e-12
        all_embs = all_embs / norms
        return all_embs

In [12]:
# Cell 3 - load data and statutes
# EDIT paths below to where your files live in Kaggle
TRAIN_PATH = "/kaggle/working/legal_dataset/train_canonical_mlb_fixed.csv"   # <-- change
DEV_PATH   = "/kaggle/working/legal_dataset/dev_canonical.csv"     # <-- change (optional)
TEST_PATH  = "/kaggle/working/legal_dataset/test_canonical.csv"    # <-- change (optional)
STATUTES_PATH = "/kaggle/working/legal_dataset/statutes.csv"       # <-- change

def load_case_csv(path: str):
    df = pd.read_csv(path, dtype=str).fillna('')
    # Expected: columns: 'text' and 'labels' or 'labels_canonical'
    if 'labels_canonical' in df.columns:
        label_col = 'labels_canonical'
    elif 'labels' in df.columns:
        label_col = 'labels'
    else:
        # try other common names
        label_col = [c for c in df.columns if 'label' in c.lower()]
        label_col = label_col[0] if label_col else None
    rows = []
    for _, r in df.iterrows():
        text = r.get('text') or r.get('case_text') or r.get('sentence') or ''
        raw_labels = r.get(label_col, '') if label_col else ''
        # normalize label string -> list of ids
        labs = []
        if isinstance(raw_labels, str) and raw_labels.strip():
            s = raw_labels.strip()
            # possible formats: "IPC_302;IPC_120B" or "['IPC_302','IPC_120B']" or "IPC_302,IPC_120B"
            if s.startswith("[") and s.endswith("]"):
                try:
                    parsed = eval(s)  # safe-ish; relies on trusted dataset
                    if isinstance(parsed, (list, tuple)):
                        labs = [str(x).strip() for x in parsed if x]
                except Exception:
                    s2 = s.strip("[]")
                    labs = [x.strip().strip("'\"") for x in re.split(r'[;,]', s2) if x.strip()]
            else:
                labs = [x.strip() for x in re.split(r'[;,]', s) if x.strip()]
        rows.append({"id": r.get('id') or r.get('case_id') or None, "text": clean_text(text), "labels": labs})
    return rows

print("Loading train/dev/test (if paths exist)...")
train = load_case_csv(TRAIN_PATH) if Path(TRAIN_PATH).exists() else []
dev = load_case_csv(DEV_PATH) if Path(DEV_PATH).exists() else []
test = load_case_csv(TEST_PATH) if Path(TEST_PATH).exists() else []

print("Train examples:", len(train), "Dev:", len(dev), "Test:", len(test))

# Load statutes.csv and build id->text
stat_df = pd.read_csv(STATUTES_PATH, dtype=str).fillna('')
# Detect id/text columns heuristically
possible_id_cols = [c for c in stat_df.columns if c.lower() in ('id','statute_id','code','section_id')]
possible_text_cols = [c for c in stat_df.columns if c.lower() in ('description','def','offense','text','definition')]
id_col = possible_id_cols[0] if possible_id_cols else stat_df.columns[0]
text_col = possible_text_cols[0] if possible_text_cols else stat_df.columns[1] if len(stat_df.columns)>1 else stat_df.columns[0]

statute_catalog = {}
for _, r in stat_df.iterrows():
    sid = str(r[id_col]).strip()
    stext = clean_text(str(r[text_col]))
    if sid:
        if not stext:
            stext = sid
        statute_catalog[sid] = stext

print("Loaded statute catalog size:", len(statute_catalog))
# If some statutes missing, we can fill them later with their ids as text

Loading train/dev/test (if paths exist)...
Train examples: 42750 Dev: 10181 Test: 13019
Loaded statute catalog size: 454


In [14]:
# ------------------ Replacement Cell 4 (Transformer with TF-IDF fallback) ------------------
# Purpose: Try to load a Hugging Face transformer embedder. If that fails (download/protobuf/401),
# fall back to a TF-IDF embedder so the pipeline proceeds without HF downloads.

from pathlib import Path
import traceback
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

class TfidfEmbedder:
    """
    Fast fallback embedder using TF-IDF + SVD (optional).
    Produces L2-normalized vectors compatible with subsequent pipeline.
    """
    def __init__(self, texts: list[str], max_features: int = 20000, use_svd: bool = False, svd_dim: int = 256):
        # texts: list of texts used to fit the TF-IDF (we'll fit using statute_texts + some queries if available)
        self.vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=max_features, stop_words='english')
        self.vectorizer.fit(texts)
        self.use_svd = use_svd
        if use_svd:
            from sklearn.decomposition import TruncatedSVD
            self.svd = TruncatedSVD(n_components=svd_dim, random_state=42)
            # we don't fit svd here; it will be fit on transform() call if needed

    def embed_texts(self, texts: list[str], show_progress: bool = False) -> np.ndarray:
        X = self.vectorizer.transform(texts).astype(np.float32)
        if self.use_svd:
            # fit-transform SVD on the fly (cheap for small data)
            X = self.svd.fit_transform(X)
        else:
            X = X.toarray()
        # L2 normalize
        norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
        X = X / norms
        return X

# Try to initialize HF Transformer embedder; if anything fails, use TF-IDF fallback
EMBED_MODEL = "distilbert-base-uncased"   # keep this default; change if you have local model path
embedder = None
use_transformer = False

# If you have a local folder containing model weights (uploaded to Kaggle dataset),
# set LOCAL_MODEL_DIR = "/kaggle/input/your-distilbert-folder"
LOCAL_MODEL_DIR = None  # set to path if you have a local HF model uploaded

try:
    # attempt to import & create transformer embedder
    if LOCAL_MODEL_DIR and Path(LOCAL_MODEL_DIR).exists():
        # local model path provided
        print("Trying to load transformer from local path:", LOCAL_MODEL_DIR)
        embedder = TransformerEmbedder(model_name=str(LOCAL_MODEL_DIR), batch_size=32)
        use_transformer = True
    else:
        print("Trying to load transformer model from HuggingFace Hub:", EMBED_MODEL)
        embedder = TransformerEmbedder(model_name=EMBED_MODEL, batch_size=32)
        use_transformer = True

    # quick smoke test: embed a tiny dummy to make sure model fully loaded
    _ = embedder.embed_texts(["test embedding"], show_progress=False)
    print("Transformer embedder ready. Using transformer embeddings.")
except Exception as e:
    print("Transformer embedder failed with exception (falling back to TF-IDF).")
    traceback.print_exc()
    # Build TF-IDF fallback using statute_texts + all case texts available (so the TF-IDF vocabulary is useful)
    all_texts_for_tfidf = []
    # statute_texts variable must exist from earlier cell (Cell 3). If not, we attempt to build minimal list.
    try:
        # statute_texts should have been created in Cell 3
        all_texts_for_tfidf.extend([t for t in statute_texts if t])
    except Exception:
        pass
    # add some query texts from train/dev/test if present
    try:
        if len(train) > 0:
            all_texts_for_tfidf.extend([ex['text'] for ex in train if ex.get('text')])
        if len(dev) > 0:
            all_texts_for_tfidf.extend([ex['text'] for ex in dev if ex.get('text')])
        if len(test) > 0:
            all_texts_for_tfidf.extend([ex['text'] for ex in test if ex.get('text')])
    except Exception:
        pass

    # ensure we have something to fit
    if not all_texts_for_tfidf:
        all_texts_for_tfidf = (statute_texts if 'statute_texts' in globals() else [" "])
    print("Fitting TF-IDF on", len(all_texts_for_tfidf), "documents")
    embedder = TfidfEmbedder(texts=all_texts_for_tfidf, max_features=20000, use_svd=False)
    use_transformer = False
    print("TF-IDF embedder ready (fallback).")

# Now compute statute embeddings using whichever embedder succeeded
statute_ids = list(statute_catalog.keys())
statute_texts = [statute_catalog[sid] for sid in statute_ids]
print("Number of statutes:", len(statute_ids))

print("Computing statute embeddings...")
statute_embs = embedder.embed_texts(statute_texts, show_progress=True)
print("Statute embeddings shape:", statute_embs.shape)

# Save artifacts
out_dir = Path("reranker_artifacts")
out_dir.mkdir(exist_ok=True)
np.save(out_dir / "statute_embs.npy", statute_embs)
with open(out_dir / "statute_ids.json", "w") as f:
    import json
    json.dump(statute_ids, f)
print("Saved statute embeddings and ids to reranker_artifacts/")

# Helpful flag: whether we used transformer (True) or TF-IDF fallback (False)
print("Used transformer embeddings:", use_transformer)

Trying to load transformer model from HuggingFace Hub: distilbert-base-uncased


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Transformer embedder ready. Using transformer embeddings.
Number of statutes: 454
Computing statute embeddings...


Embedding:   0%|          | 0/15 [00:00<?, ?it/s]

Statute embeddings shape: (454, 768)
Saved statute embeddings and ids to reranker_artifacts/
Used transformer embeddings: True


In [15]:
# Cell 5 - dense retrieval candidate generation
def retrieve_topk_dense(query: str, query_emb: np.ndarray, statute_embs: np.ndarray, topk: int = 50):
    """
    query_emb: normalized vector shape (D,) or (1,D)
    statute_embs: normalized (N, D)
    returns list of (idx, score) sorted desc
    """
    # dot-product since embeddings are normalized => cosine
    scores = np.dot(statute_embs, query_emb.reshape(-1))
    topk_idx = np.argsort(scores)[::-1][:topk]
    return [(int(i), float(scores[i])) for i in topk_idx]

# convenience: embed a single query
def embed_single_query(text: str):
    embs = embedder.embed_texts([text], show_progress=False)
    return embs[0]

In [16]:
# Cell 6 - create pairwise training dataset
def create_pairwise_examples(cases: List[dict], statute_ids: List[str], statute_embs: np.ndarray, topk=50, neg_per_pos=3):
    """
    Return list of dicts: {'query_text':..., 'q_emb':np.array, 'cand_idx':int, 'cand_emb':np.array, 'label':0/1}
    """
    examples = []
    for ex in tqdm(cases, desc="build_pairs"):
        qtext = ex['text']
        gold = set([g for g in ex['labels'] if g in statute_catalog])
        if not qtext:
            continue
        q_emb = embed_single_query(qtext)
        # retrieve topk
        cand_idx_scores = retrieve_topk_dense(qtext, q_emb, statute_embs, topk=topk)
        cand_indices = [i for i,_ in cand_idx_scores]
        # positives among retrieved
        pos_indices = [i for i in cand_indices if statute_ids[i] in gold]
        # if no positives among topk, still add one positive if some gold in catalog (helps training)
        if not pos_indices and gold:
            for g in gold:
                if g in statute_ids:
                    pos_indices.append(statute_ids.index(g))
        # negatives pool
        neg_pool = [i for i in cand_indices if i not in pos_indices]
        # sample negatives
        for p in pos_indices:
            examples.append({"q_text": qtext, "q_emb": q_emb, "cand_idx": p, "cand_emb": statute_embs[p], "label": 1})
            # sample negatives for this positive
            n_take = min(len(neg_pool), neg_per_pos)
            if n_take > 0:
                sampled = random.sample(neg_pool, n_take)
                for ni in sampled:
                    examples.append({"q_text": qtext, "q_emb": q_emb, "cand_idx": ni, "cand_emb": statute_embs[ni], "label": 0})
    random.shuffle(examples)
    return examples

print("Creating training pairs from train split...")
train_pairs = create_pairwise_examples(train, statute_ids, statute_embs, topk=50, neg_per_pos=3)
print("Train pairs:", len(train_pairs))

Creating training pairs from train split...


build_pairs:   0%|          | 0/42750 [00:00<?, ?it/s]

Train pairs: 439004


In [18]:
# Cell 7 - dataset and dataloader
class PairwiseDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        it = self.pairs[idx]
        # return numpy arrays; convert to float32
        return {
            "q_emb": it["q_emb"].astype(np.float32),
            "cand_emb": it["cand_emb"].astype(np.float32),
            "label": float(it["label"])
        }

def collate_pairs(batch):
    q_emb = torch.tensor([b["q_emb"] for b in batch], dtype=torch.float32)
    cand_emb = torch.tensor([b["cand_emb"] for b in batch], dtype=torch.float32)
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.float32).unsqueeze(1)
    return q_emb.to(DEVICE), cand_emb.to(DEVICE), labels.to(DEVICE)

batch_size = 64
train_dataset = PairwiseDataset(train_pairs)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_pairs)

In [19]:
# Cell 8 - MLP reranker model
class MLP_Reranker(nn.Module):
    def __init__(self, emb_dim: int, hidden_dim: int = 512, dropout: float = 0.2):
        super().__init__()
        # Input is [q_emb, cand_emb, abs_diff, elementwise_prod] concatenated
        in_dim = emb_dim * 4
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim//2, 1)  # output score (logit)
        )

    def forward(self, q_emb, cand_emb):
        # q_emb, cand_emb: (B, D)
        diff = torch.abs(q_emb - cand_emb)
        prod = q_emb * cand_emb
        x = torch.cat([q_emb, cand_emb, diff, prod], dim=1)
        out = self.net(x)
        return out

emb_dim = statute_embs.shape[1]
model = MLP_Reranker(emb_dim, hidden_dim=512, dropout=0.2).to(DEVICE)
print(model)

MLP_Reranker(
  (net): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=256, out_features=1, bias=True)
  )
)


In [20]:
# Cell 9 - training loop
import torch.optim as optim
from sklearn.metrics import roc_auc_score

def train_reranker(model, train_loader, epochs=6, lr=2e-4, val_loader=None):
    opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.BCEWithLogitsLoss()
    best_loss = 1e9
    for ep in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        for q_emb, cand_emb, labels in tqdm(train_loader, desc=f"Train ep{ep}"):
            opt.zero_grad()
            logits = model(q_emb, cand_emb)
            loss = criterion(logits, labels)
            loss.backward()
            opt.step()
            total_loss += loss.item() * q_emb.size(0)
        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {ep} train_loss: {avg_loss:.6f}")
        # optional: evaluate on small validation set for monitoring
    return model

# Train (adjust epochs based on resources)
model = train_reranker(model, train_loader, epochs=6, lr=2e-4)
# Save model
torch.save(model.state_dict(), out_dir / "mlp_reranker_state.pth")
print("Saved reranker state.")

Train ep1:   0%|          | 0/6860 [00:00<?, ?it/s]

/tmp/ipykernel_47/3348135988.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  q_emb = torch.tensor([b["q_emb"] for b in batch], dtype=torch.float32)


Epoch 1 train_loss: 0.124271


Train ep2:   0%|          | 0/6860 [00:00<?, ?it/s]

Epoch 2 train_loss: 0.077559


...

Epoch 4 train_loss: 0.066755


Train ep5:   0%|          | 0/6860 [00:00<?, ?it/s]

Epoch 5 train_loss: 0.063722


Train ep6:   0%|          | 0/6860 [00:00<?, ?it/s]

Epoch 6 train_loss: 0.062060
Saved reranker state.


In [21]:
# Cell 10 - evaluation utilities
def reranker_score_for_query(model, q_emb: np.ndarray, candidate_indices: List[int]):
    """
    Given a single query embedding and list of candidate indices, compute reranker scores.
    Returns list of (idx, score)
    """
    model.eval()
    qs = np.repeat(q_emb.reshape(1,-1), len(candidate_indices), axis=0).astype(np.float32)
    cs = statute_embs[candidate_indices].astype(np.float32)
    with torch.no_grad():
        q_t = torch.tensor(qs, dtype=torch.float32).to(DEVICE)
        c_t = torch.tensor(cs, dtype=torch.float32).to(DEVICE)
        logits = model(q_t, c_t).cpu().numpy().reshape(-1)
    return list(zip(candidate_indices, logits.tolist()))

def evaluate_split(split_cases: List[dict], statute_ids: List[str], statute_embs: np.ndarray,
                   model, topk=100, rerank_k=10):
    recall_hits = 0
    rr_total = 0.0
    count_q = 0
    for ex in tqdm(split_cases, desc="Eval"):
        qtext = ex['text']
        gold = set([g for g in ex['labels'] if g in statute_catalog])
        if not qtext or len(gold)==0:
            continue
        count_q += 1
        q_emb = embed_single_query(qtext)
        cand_idx_scores = retrieve_topk_dense(qtext, q_emb, statute_embs, topk=topk)
        cand_indices = [i for i,_ in cand_idx_scores]
        # rerank using model
        reranked = reranker_score_for_query(model, q_emb, cand_indices)
        # sort by score desc
        ranked = sorted(reranked, key=lambda x: x[1], reverse=True)
        topk_ranked = [statute_ids[idx] for idx,_ in ranked[:rerank_k]]
        if any(sid in gold for sid in topk_ranked):
            recall_hits += 1
        # compute reciprocal rank
        rr = 0.0
        for rank_pos, (idx, _) in enumerate(ranked, start=1):
            sid = statute_ids[idx]
            if sid in gold:
                rr = 1.0 / rank_pos
                break
        rr_total += rr
    recall_at_k = recall_hits / count_q if count_q>0 else 0.0
    mrr = rr_total / count_q if count_q>0 else 0.0
    return {"recall@{}".format(rerank_k): recall_at_k, "mrr": mrr, "n_queries": count_q}

# Evaluate on dev and test (if present)
if len(dev)>0:
    dev_metrics = evaluate_split(dev, statute_ids, statute_embs, model, topk=100, rerank_k=10)
    print("Dev metrics:", dev_metrics)
if len(test)>0:
    test_metrics = evaluate_split(test, statute_ids, statute_embs, model, topk=100, rerank_k=10)
    print("Test metrics:", test_metrics)

Eval:   0%|          | 0/10181 [00:00<?, ?it/s]

Dev metrics: {'recall@10': 0.390479000688502, 'mrr': 0.21090147599864018, 'n_queries': 10167}


Eval:   0%|          | 0/13019 [00:00<?, ?it/s]

Test metrics: {'recall@10': 0.4097831102907245, 'mrr': 0.21605110016625156, 'n_queries': 13002}


In [24]:
# Cell 11 - save artifacts and demo
# Save tokenizer/model metadata (we reuse embedding model; saving reranker separately)
torch.save({
    "statute_ids": statute_ids,
    "statute_texts": statute_texts
}, out_dir / "statute_catalog.pt")

# Save embedder model name
with open(out_dir / "embedder_info.json", "w") as f:
    json.dump({"embed_model": EMBED_MODEL, "emb_dim": int(statute_embs.shape[1])}, f)

print("Artifacts saved in", out_dir)

# Demo inference function
def predict_for_text(query: str, topk=20, rerank_k=10):
    q_emb = embed_single_query(query)
    dense_candidates = retrieve_topk_dense(query, q_emb, statute_embs, topk=topk)
    cand_indices = [i for i,_ in dense_candidates]
    reranked = reranker_score_for_query(model, q_emb, cand_indices)
    ranked = sorted(reranked, key=lambda x: x[1], reverse=True)
    out = []
    for idx, score in ranked[:rerank_k]:
        out.append({"statute_id": statute_ids[idx], "statute_text": statute_catalog[statute_ids[idx]], "score": float(score)})
    return out

# Example
if len(train)>0:
    sample_q = train[0]['text']
    print("Sample query (truncated):", sample_q[:400])
    preds = predict_for_text(sample_q, topk=50, rerank_k=10)
    for p in preds:
        print(p['statute_id'], " score:", p['score'])

Artifacts saved in reranker_artifacts
Sample query (truncated): ['(a), <SECTION> of the <ACT>. The gist of theprosecution case relevant <ENTITY> the purpose of this proceedingmay be stated thus: With the growth of industry, commerceand trade in and around the city of Mumbai which generatessubstantial quantity of wealth, there has been increase oforganised activities by gangs of anti-socials to extractmoney from affluent sections of society like developers,hote
IPC_376  score: -1.1044032573699951
IPC_304B  score: -1.5030287504196167
IPC_195  score: -4.679609298706055
IPC_123  score: -4.722894668579102
IPC_376A  score: -5.146350860595703
IPC_170  score: -5.626493453979492
IPC_127  score: -5.887179374694824
IPC_418  score: -6.628131866455078
IPC_194  score: -7.077974796295166
IPC_205  score: -7.2961835861206055
